In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import time
from ratelimit import limits, sleep_and_retry

# The main.py file is the main code body for the PyBuoy application.
#note: captain_seemore is available as a repo name on GitHub

# Helper functions
@sleep_and_retry
def get_drifting_buoy_data(buoy_id):
    # Get buoy data from NOAA # https://www.ndbc.noaa.gov/data/realtime2/{buoy_id}.drift
    url = f'https://www.ndbc.noaa.gov/data/realtime2/{buoy_id}.drift' # f-string url
    r = requests.get(url) # Get the data from the URL
    with open('data.csv', 'wb') as f: # Write the data to a file
        f.write(r.content) # Write the data to a file
    df = pd.read_csv('data.csv', header=1, parse_dates=True, delimiter = '\s+') # Read in the data
    return df # return the buoy data

@sleep_and_retry
def get_stationary_buoy_data(buoy_id):
    # https://www.ndbc.noaa.gov/data/realtime2/21415.dart
    # example buoy data pull for water column height (capture the latest one)
    url = f'https://www.ndbc.noaa.gov/data/realtime2/{buoy_id}.dart' # f-string url
    r = requests.get(url)
    with open('data.csv', 'wb') as f:
        f.write(r.content)
    df = pd.read_csv('data.csv', header=1, parse_dates=True, delimiter = '\s+')
    df.head()
    return df


# Buoy Cam Data
def get_buoy_cam(buoy_id):
    """
    NDBC operates BuoyCAMs at several stations. These BuoyCAMs typically take photos only during daylight hours.

    To view the most recent BuoyCAM image from an NDBC station, use this URL:

        https://www.ndbc.noaa.gov/buoycam.php?station=xxxxx

    where xxxxx is the desired station ID. To see which stations are currently reporting BuoyCAM images, check the BuoyCAMs map.

    If the server encounters any difficulties in processing your request, you will receive one of these error messages:

        No station specified

        Modify your URL to use the station parameter to specify a valid station with a BuoyCAM (station=xxxxx where xxxxx is the station ID). Review the BuoyCAMs map to see which stations have a BuoyCAM.
        Invalid station specified

        The station on the URL is not a valid station. Review the BuoyCAMs map to see which stations have a BuoyCAM.
        This station has no BuoyCAM

        The station on the URL is valid but has no BuoyCAM installed. Look at the BuoyCAMs map to see which stations have a BuoyCAM.
        BuoyCAM photo for this station is older than 16 hours

        The BuoyCAM on the specified station has not reported in the past 16 hours, hence there is no image to display.
        Unable to access BuoyCAMs at this time

        There is an issue preventing the BuoyCAM process from functioning properly. Recommend waiting at least 30 minutes and trying again, if the problem persist contact the NDBC webmaster with the URL used and the date/time the error was received."""

    # https://www.ndbc.noaa.gov/buoycam.php?station=21415
    url = f'https://www.ndbc.noaa.gov/buoycam.php?station={buoy_id}' # f-string url
    r = requests.get(url) # Get the data from the URL
    r_text = r.text # get the text from the request
    print(r_text)

    with open('data.csv', 'wb') as f:
        f.write(r.content) # this writes in latin-1 encoding
    df = pd.read_csv('data.csv', header=1, parse_dates=True, delimiter = '\s+') # Read in the data
    df.head() # this is a pandas dataframe
    return df # Main code body



def get_available_buoy_ids():

    # source for this data: https://www.ndbc.noaa.gov/to_station.shtml

    # task 1 -  use regex to extract all numbers like 44004 from stations.txt file and save to a list.
    # task 2 - use regex to also identify any identifiers like ALXN6 or ALXN7 and append them to the list as well.
    import re
    stations_text = open('data/stations.txt', 'r').read()
    task_one_stations_list = re.findall(r' \d+ ', stations_text) # find all numbers in the file
    task_two_stations_list = re.findall(r' \w+\d ', stations_text) # find all identifiers in the file
    # combine by adding the second list to the first with .extend()
    task_one_stations_list.extend(task_two_stations_list)
    # remove the spaces from the list
    stations_list = [x.strip() for x in task_one_stations_list] # remove whitespace
    # strip the whitespace from the list
    stations_list = [x.strip() for x in stations_list] # remove whitespace
    # remove duplicates
    stations_list = [x for x in task_one_stations_list if x != ''] # remove empty strings
    stations_list = list(dict.fromkeys(stations_list))
    # remove the first element

    # remove all nonalpha characters from the elements in the list
    stations_list = [re.sub(r'\W+', '', x) for x in stations_list]
    print(len(stations_list), ' stations were identified')
    # print(stations_list)


    return stations_list



In [ ]:
def main():
    # First we need to get the list of available buoy ids
    # Then we need to ask the question, "Does this buoy transmit photos? (BuoyCam) Y/N"
    # If it does then we want to get the latest photo and save it to the data folder with the buoy id as the file name (every time we run this script we want to overwrite the existing photo with the latest one.)

    buoy_ids = get_available_buoy_ids()
    #* Now we have a list of buoy ids that we can use to get photos from (potentially).
    #? Question: How do we know if a buoy has a camera? (BuoyCam)
    # https://www.ndbc.noaa.gov/buoycam.php?station=XXXXX

    # We have to check the buoy id to see if it has a BuoyCam if it does not then we will see an error message: "This station has no BuoyCAM"
    # The errors that merit dropping the buoy id are:
    # an error that contains: "There is an issue preventing the BuoyCAM process"
    # an error that contains: "URL is valid but has no BuoyCAM installed"
    # an error that contains: "BuoyCAM photo for this station is older than 16 hours"

    # other errors just mean that the buoy is not transmitting data at the moment and we can try again later.

    # Process Flow:
    # 1. Get the list of buoy ids
    # 2. For each buoy id:
    # 3. Check if the buoy has a BuoyCam
    # 4. If it does...
    # 4a. The response from the request has an encoding that will need to be converted to utf-8.
        #then get the latest photo and save it to the data folder with the buoy id as the file name (every time we run this script we want to overwrite the existing photo with the latest one.)



main()